# Load optimisation
## Dr Jose M. Albornoz
### January 2024

Taken from [this blog post](https://medium.com/thelorry-product-tech-data/load-optimization-problem-1baa116656df). I have done a bit of code cleaning and organisation for easier readability.

The purpose of packing problems is to determine the optimum way to pack a collection of items with varying sizes into containers with defined capacities. A typical application is efficiently putting boxes onto delivery vehicles. Due to capacity limits, it is frequently impossible to pack all of the products. In that case, the problem is to determine a subset of the items with the most significant overall size that will fit in the containers.

There are many types of packing problems. Knapsack problems and bin packing are two of the most common.

**Knapsack problem**

The knapsack problem is a combinatorial optimization problem in which you must decide how many items should be included in a container. The overall weight of the collection must be less than or equal to a specific limit. The problem derives its name from the dilemma faced by someone who is constrained by a fixed-size knapsack and must fill it with the most valuable items.

The issue frequently arises in resource allocation when decision-makers must choose among various non-divisible projects or tasks under a defined budget or time limit.

**Bin packing problem**

Similar to the knapsack problem, the bin packing issue is also an optimization problem. A finite number of bins or containers, each with a defined capacity, must be packed to minimize the total number of bins used.

Filling containers, loading vehicles with weight capacity limits, making file backups in media, technology mapping in FPGA semiconductor chip design, putting boxes onto delivery vehicles are all examples of the problem.

**The problem at hand**

We need to load a number of items with certain weights and volumes onto a number of vehicles, each with a given maximum volume and weight capacity. The loading must be done in the most efficient way so that the unused capacity for each vehicle is minimised.

We will show how this problem can be solved using [Google's OR-Tools library](https://developers.google.com/optimization). 
OR-Tools is open source software for combinatorial optimization, able to find the best solution to a problem out of an extensive set of possible solutions. Here are some examples of problems that OR-Tools addresses:

* Vehicle routing: Find optimal routes for vehicle fleets that pick up and deliver packages given constraints.
* Scheduling: Find the optimal schedule for a complex set of tasks that need to be performed before others on a fixed set of machines or other resources.
* Bin packing: Pack as many objects of various sizes as possible into a fixed number of bins with maximum capacities.

Problems like these typically have many viable solutions — too numerous for a computer to sort through. OR-Tools overcomes this by narrowing the search set with cutting-edge algorithms to obtain an optimal (or near-optimal) answer.

# 0.- Imports

In [1]:
import pandas as pd
from ortools.linear_solver import pywraplp

RANDOM_STATE = 801
pd.options.mode.chained_assignment = None

# maximum number of rdataframe ows and columns displayed
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 500)

import warnings
warnings.filterwarnings('ignore')

# 1.- Load data for items to be shipped

In [2]:
data_path = "data/examples.csv"
items_to_be_shipped = pd.read_csv(data_path)
items_to_be_shipped.head()

,Actual Weight,Actual Width,Actual Height,Actual Length
0,5.7,45.0,1.0,81.0
1,19.0,38.0,39.0,85.0
2,20.0,5.0,8.0,6.0
3,36.0,45.0,95.0,48.0
4,10.0,15.0,15.0,82.0


The data consist of the actual weight, width, height, and length for each of 367 items. The actual width, height, and length will be used to calculate the volume of the item. The Actual Weight is in kilograms (kg), while the item's dimensions are in centimetres (cm).

## 1.1.- Compute volume of items

In [3]:
items_to_be_shipped['Volume'] = items_to_be_shipped['Actual Weight'] * items_to_be_shipped['Actual Height'] * \
                                items_to_be_shipped['Actual Length']
items_to_be_shipped.head()

,Actual Weight,Actual Width,Actual Height,Actual Length,Volume
0,5.7,45.0,1.0,81.0,461.7
1,19.0,38.0,39.0,85.0,62985.0
2,20.0,5.0,8.0,6.0,960.0
3,36.0,45.0,95.0,48.0,164160.0
4,10.0,15.0,15.0,82.0,12300.0


## 1.2.- Total volume and weight

In [4]:
total_volume = items_to_be_shipped['Volume'].sum()
total_weight = items_to_be_shipped['Actual Weight'].sum()

print(f'Total Volume: {round(total_volume,)} cm3')
print(f'Total Volume: {round(total_volume*1e-6, 2)} m3')
print(f'Total Weight: {round(total_weight, 2)} kg')

Total Volume: 46254303.0 cm3
Total Volume: 46.25 m3
Total Weight: 9853.69 kg


Our problem: for this total volume and weight, how many vehicles should be used so that vehicle capacity usage is maximised?

## 1.3.- Delivery vehicles

We will now define the number of vehicles at our disposal and their capacity. Each item in the list defined below contains the following fields:

* code: an identifier for a vehicle
* number: number of vehicles available
* max_weight: vehicle's maximum weight in kilograms
* max_volume: vehicle's maximum volume in cubic centimetres

In [5]:
availableVehicles = [
    {'code': 'LORRY-L',
    'number': 1,
    'max_weight': 5000,
    'max_volume': 24261874.16},
    {'code': 'LORRY-M',
    'number': 2,
    'max_weight': 3000,
    'max_volume': 19980366.96},
    {'code': 'LORRY-S',
    'number': 3,
    'max_weight': 1000,
    'max_volume': 7079211.65},
    {'code': 'VAN', 
     'number': 3, 
     'max_weight': 500, 
     'max_volume': 2378615.11},
    {'code': 
     '4x4', 
     'number': 6, 
     'max_weight': 500, 
     'max_volume': 1189307.56}
]

In [6]:
availableVehicles.reverse()

In [7]:
availableVehicles

[{'code': '4x4', 'number': 6, 'max_weight': 500, 'max_volume': 1189307.56},
 {'code': 'VAN', 'number': 3, 'max_weight': 500, 'max_volume': 2378615.11},
 {'code': 'LORRY-S',
  'number': 3,
  'max_weight': 1000,
  'max_volume': 7079211.65},
 {'code': 'LORRY-M',
  'number': 2,
  'max_weight': 3000,
  'max_volume': 19980366.96},
 {'code': 'LORRY-L',
  'number': 1,
  'max_weight': 5000,
  'max_volume': 24261874.16}]

# 2.- Define data structure for our problem

In [8]:
def create_data_model(shippedItems, availableVehicles):
    
    """
       Create a data dictionary for optimisation.
    
       parameters:
       * shippedItems: dataframe containing weight and volume for each item to be shipped
       * availableVehicles: list of dictionaries, each containing a vehicle's identifier, 
         number of available vehicles of a given type, maximum weight capacity anc maximum
         volume capacity
    
       The data output (data) is a dictionary containing:
       
       * weights: each item's weight
       * volumes: each items volume
       * items: numerical identifier for each item
       * num_items: number of items to be shipped
       * max_volume: maximum volume that can be carried by each vehicle
       * max_weight: maximum weight that can be carried by each vehicle
       * truck_types: type of each vehicle
       * trucks: numerical identifier for each truck
    
    """
    data = {}
    weights = shippedItems['Actual Weight'].to_list()
    volumes = shippedItems['Volume'].to_list()
    
    data['weights'] = weights
    data['volumes'] = volumes
    
    data['items'] = list(range(len(weights)))
    data['num_items'] = len(weights)
    
    max_volumes = []
    max_weights = []
    truck_types = []
    
    # reverse available vehicle data so that we start from smaller vehicles first
    availableVehicles.reverse()
    
    # register max_weight and max_volume for each available vehicle
    for tL in availableVehicles:
        for i in range(tL['number']):
            max_volumes.append(tL['max_volume'])
            max_weights.append(tL['max_weight'])
            truck_types.append(tL['code'])
    
    data['max_volume'] = max_volumes 
    data['max_weight'] = max_weights 
    data['truck_types'] = truck_types
    
    data['trucks'] = list(range(len(data['max_volume'])))
    
    return data

In [9]:
data = create_data_model(items_to_be_shipped, availableVehicles)

# 3.- Define solver

In [11]:
# Create mip solver with the SCIP backend.
solver = pywraplp.Solver.CreateSolver('SCIP')

## 3.1.- Define variables

In [12]:
# Variables
# x[i, j] = 1 if item i is placed in vehicle j.
x = {}
for i in data['items']:
    for j in data['trucks']:
        x[(i, j)] = solver.IntVar(0, 1, 'x_%i_%i' % (i, j))

## 3.2.- Define constraints

In [13]:
# Each item can be in at most one vehicle.
for i in data['items']:
    solver.Add(sum(x[i, j] for j in data['trucks']) <= 1)
    
# The amount packed in each vehicle cannot exceed its max weight.
for j in data['trucks']:
    solver.Add(sum(x[(i, j)] * data['weights'][i] for i in data['items']) <= data['max_weight'][j])
    
# The amount packed in each vehicle cannot exceed its max volume.
for j in data['trucks']:
    solver.Add(sum(x[(i, j)] * data['volumes'][i] for i in data['items']) <= data['max_volume'][j])

## 3.3.- Add objectives

In [14]:
objective = solver.Objective()

for i in data['items']:
    for j in data['trucks']:
        objective.SetCoefficient(x[(i, j)], data['volumes'][i])
        
objective.SetMaximization()

## 3.4.- Solve problem

In [15]:
status = solver.Solve()

In [16]:
status

0

# 4.- Present solution

In [17]:
_totalLeftVolume = 0
_totalLeftWeight = 0

if status == pywraplp.Solver.OPTIMAL:
    
    assign = []
    total_weight = 0
    total_items = 0
    
    print('Total vehicle: ')
    display(availableVehicles)
    print()
    print('Total Items:', len(items_to_be_shipped))
    print()
    
    for j in data['trucks']:
        
        # weight and volume per vehicle
        bin_weight = 0
        bin_volume = 0
        
        print('Truck ', j, '[', data['truck_types'][j] ,'] - \
               max_weight:[', "{:,.2f}".format(data['max_weight'][j]), '] \
               - max volume:[', "{:,.2f}".format(data['max_volume'][j]), ']')
        
        for i in data['items']:
            
            if x[i, j].solution_value() > 0:
                assign.append(i)
                total_items += 1
                print('Item', i, '- weight:', data['weights'][i], ' volumes:', data['volumes'][i])
                bin_weight += data['weights'][i]
                bin_volume += data['volumes'][i]
                
        print('Packed truck volume:', "{:,.2f}".format(bin_volume))
        print('Packed truck weight:', "{:,.2f}".format(bin_weight))
        print()
        
        # computes unused vehicle capacity
        if (bin_volume > 0) & (bin_weight > 0):
            leftVolume = data['max_volume'][j] - bin_volume
            leftWeight = data['max_weight'][j] - bin_weight
        else:
            leftVolume = 0
            leftWeight = 0
            
        print('Unused Volume', "{:,.2f}".format(leftVolume))
        print('Unused Weight', "{:,.2f}".format(leftWeight))
        print()
        print()
        
        total_weight += bin_weight
        _totalLeftVolume += leftVolume
        _totalLeftWeight += leftWeight
        
    print('Total packed weight:', "{:,.2f}".format(total_weight))
    print('Total packed volume:', "{:,.2f}".format(objective.Value()))
    print('Total item assigned:', "{:,.0f}".format(total_items))
    print()
    print("#" * 70)
    print('Total Left Volume', "{:,.2f}".format(_totalLeftVolume))
    print('Total Left Weight', "{:,.2f}".format(_totalLeftWeight))
    print("#" * 70)
    
else:
    
    print('The problem does not have an optimal solution.')
    print()

Total vehicle: 


[{'code': 'LORRY-L',
  'number': 1,
  'max_weight': 5000,
  'max_volume': 24261874.16},
 {'code': 'LORRY-M',
  'number': 2,
  'max_weight': 3000,
  'max_volume': 19980366.96},
 {'code': 'LORRY-S',
  'number': 3,
  'max_weight': 1000,
  'max_volume': 7079211.65},
 {'code': 'VAN', 'number': 3, 'max_weight': 500, 'max_volume': 2378615.11},
 {'code': '4x4', 'number': 6, 'max_weight': 500, 'max_volume': 1189307.56}]


Total Items: 367

Truck  0 [ LORRY-L ] -                max_weight:[ 5,000.00 ]                - max volume:[ 24,261,874.16 ]
Item 0 - weight: 5.7  volumes: 461.7
Item 2 - weight: 20.0  volumes: 960.0
Item 6 - weight: 65.0  volumes: 715000.0
Item 7 - weight: 13.9  volumes: 81871.0
Item 8 - weight: 18.0  volumes: 45360.0
Item 9 - weight: 2.5  volumes: 3375.0
Item 10 - weight: 30.0  volumes: 72000.0
Item 11 - weight: 18.0  volumes: 69678.0
Item 12 - weight: 6.8  volumes: 3046.4
Item 13 - weight: 55.0  volumes: 304920.0
Item 14 - weight: 25.0  volumes: 6000.0
Item 15 - weight: 1.35  volumes: 1012.5
Item 16 - weight: 7.0  volumes: 5264.0
Item 17 - weight: 6.7  volumes: 6700.0
Item 18 - weight: 3.0  volumes: 3240.0
Item 19 - weight: 39.0  volumes: 237120.0
Item 20 - weight: 4.0  volumes: 6660.0
Item 21 - weight: 69.0  volumes: 557175.0
Item 22 - weight: 30.0  volumes: 113400.0
Item 23 - weight: 15.0  volumes: 63000.0
Item 24 - weight: 6.0  volumes: 11250.0
Item 25 - weight: 29.9  volumes: 